##### Import required Libraries

In [ ]:
import os 
import tensorflow as tf 
from google.colab import drive
import matplotlib.pyplot as plt
from tensorflow.keras import layers 
from tensorflow.keras.applications.resnet50 import ResNet50 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
drive.mount('/content/gdrive')

##### Check if GPU is detected

In [ ]:
device_name = tf.test.gpu_device_name()
print(device_name)
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

##### Split the dataset into train and validation

In [ ]:
base_dir = '/content/gdrive/MyDrive/FYP/binary_classification_data/binary_classification_data'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
train_normal_dir = os.path.join(train_dir, 'normal')
train_abnormal_dir = os.path.join(train_dir, 'abnormal')
validation_normal_dir = os.path.join(validation_dir, 'normal')
validation_abnormal_dir = os.path.join(validation_dir, 'abnormal')

##### Augment and generate images

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
valid_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 20, class_mode = 'binary', target_size = (512, 512))
validation_generator = valid_datagen.flow_from_directory(validation_dir,  batch_size = 20, class_mode = 'binary', target_size = (512, 512))

##### Defining base model and unfreezing layers

In [ ]:
base_model = ResNet50(input_shape = (512, 512, 3),
                   include_top = False,
                   weights = 'imagenet')

trainable_blocks = ['block5_conv1', 'block5_conv2', 'block5_conv3']

for layer in base_model.layers:
    if layer.name not in trainable_blocks:
        layer.trainable = False
    else:
        layer.trainable = True

base_model.summary()

##### Adding projection layers to the backbone network

In [ ]:
x = layers.Flatten()(base_model.output)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.models.Model(base_model.input, x)
model.summary()

##### Compiling the model and defining the optimizer, loss function, metrics

In [ ]:
with tf.device('/device:GPU:0'):
  model = tf.keras.models.Model(base_model.input, x)
  model.compile(
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001),
        loss = 'binary_crossentropy',
        metrics = 
            ['acc',
              tf.keras.metrics.Precision(name='precision'),
              tf.keras.metrics.Recall(name='recall')
            ]
      )
  reshist = model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 1)

##### Plotting train and validation results

In [ ]:
acc = reshist.history['acc']
val_acc = reshist.history['val_acc']
loss = reshist.history['loss']
val_loss = reshist.history['val_loss']
epochs_range = range(10)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

##### Evaluating the model on the test dataset

In [ ]:
test_dir = os.path.join(base_dir, 'test')
test_normal_dir = os.path.join(test_dir, 'normal')
test_abnormal_dir = os.path.join(test_dir, 'abnormal')

test_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_generator = test_datagen.flow_from_directory(test_dir,  batch_size = 20, class_mode = 'binary', target_size = (512, 512))

model.evaluate(test_generator)